# Assignment 2

 Shabbir-301383831
 Importing and defining variables, Function to write output data, to get top 100 movie reviews and to get top 100 reviews for each of top 1000 movies.
 
Muhammad-301420568
Function to analyze sentiment, to get the sentiment score for each review of movie and for each movie and function to get number of positve and negative reviews.

Combined work between both of us to generate the output and interpret the results.
 
 Sentiment Analysis on movie reviews of the top 250 movies on IMDB, using the NLTK VADER method, we have used polarity scores of each word to judge the review.

### References
 - Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

#### Importing Required Packages

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import pandas as pd
import csv

#### Defining Global Variables

In [10]:
# Download NLTK data
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vatsaal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Vatsaal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### Defining Utility Functions

Theses are the utility functions used in this notebook for the analysis.

In [15]:
# Function to write the output data to a CSV file
def writeDataCSV(data):
    try:
        with open("output.csv", 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames= ['movies', 'negative', 'positive'])
            writer.writeheader()
            for item in data:
                writer.writerow(item)
    except IOError:
        print("I/O Error")
        
# Function to get the top 100 reviews for a given movie
def get_reviews(movie_id):
    reviews = []
    for i in range(1, 2):
        url = f"https://www.imdb.com/title/{movie_id}/reviews/_ajax?sort=submissionDate&dir=desc&ratingFilter=0&page={i}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        reviews.extend([r.text.strip() for r in soup.select(".content .text")])
        if len(reviews) >= 5:
            break
    return reviews


# Function to get the top 100 reviews for each of the top 1000 movies
def get_movie_reviews():
    url = "https://www.imdb.com/chart/top"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    movies = soup.select(".lister-list tr")
    movie_reviews = {}
    for i in range(1, len(movies)):
        movie_id = re.search("/title/(.*)/", movies[i].select_one(".titleColumn a")["href"]).group(1)
        movie_name = movies[i].select_one(".titleColumn a").text
        movie_reviews[movie_name] = get_reviews(movie_id)
    return movie_reviews

# Function to analyze the sentiment of a given text
def analyze_sentiment(text):
    stop_words = set(stopwords.words('english'))
    sia = SentimentIntensityAnalyzer()
    words = text.lower().split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    return sia.polarity_scores(text)['compound']


# Function to get the sentiment score for each review of a movie
def get_sentiment_scores(reviews):
    scores = []
    for review in reviews:
        scores.append(analyze_sentiment(review))
    return scores

# Function to get the sentiment scores for each review of each movie
def get_movie_sentiment_scores(movie_reviews):
    movie_sentiment_scores = {}
    for movie, reviews in movie_reviews.items():
        movie_sentiment_scores[movie] = get_sentiment_scores(reviews)
    return movie_sentiment_scores

# Function to get the number of positive and negative reviews for each movie
def get_movie_sentiment_counts(movie_sentiment_scores):
    movie_sentiment_counts = {}
    for movie, scores in movie_sentiment_scores.items():
        positive = sum(1 for score in scores if score > 0)
        negative = sum(1 for score in scores if score < 0)
        movie_sentiment_counts[movie] = {"Positive": positive, "Negative": negative}
    return movie_sentiment_counts


#### Using all the functions above to generate the ouput

In [17]:
# First we will fetch all the movies and their respective reviews
movie_reviews = get_movie_reviews()

# Analysing each review using their scores
movie_sentiment_scores = get_movie_sentiment_scores(movie_reviews)

# Counting each reviews to create the output Data
movie_sentiment_counts = get_movie_sentiment_counts(movie_sentiment_scores)

# Creating a Pandas Dataframe to view the output Data in a formatted way
movie_sentiment_counts_df = pd.DataFrame(movie_sentiment_counts).T

# # Writing the data to a CSV file "output.csv"
# writeDataCSV(movie_sentiment_counts)

# Dataframe preview
print("Result")
movie_sentiment_counts_df

Result


,Positive,Negative
The Godfather,23,2
The Dark Knight,22,3
The Godfather Part II,23,2
12 Angry Men,20,5
Schindler's List,20,5
...,...,...
The Help,22,3
Dersu Uzala,24,1
Aladdin,25,0
Dances with Wolves,21,4
